### Problems: 
#### Longitudinal data of young children, we need to check if we take one entry each ? then we are only left with a few... 
#### subject ID as actually the match group (.xlsx)

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os, sys
from scipy.io import mmread
sys.path.append(os.path.join(os.getcwd(), '..', 'utils'))
from filepath import *
%load_ext autoreload
%autoreload 2

In [ ]:
#NOTE: There is a xlsx file in the folder that contains some info i try to integrate into the df´s
### NOTE: The first merged dataset here is not the one refered in the paper. the GSE2397987 one is found as the second one merged here... 
# Its a bit weird as the PBMC sample in GEO got only 86 samples and this one 119 (123) 


In [ ]:
# what is disStage
### disease stage: paper specific
# what is Diff.From.Infant.Pos
### paper specific
# why is counts 119 and metadata 123 ?  
### See .xlsx file: 3 are excluded  

metadata = pd.read_csv(PATH_WIMMER2023_METADATA, sep=",")
print(metadata.shape)
metadata.head()


In [ ]:
rawdata = pd.read_csv(PATH_WIMMER2023_RAWDATA, sep="\t", header=None).T
rawdata.columns = rawdata.iloc[0]
rawdata = rawdata.iloc[1:]
print(rawdata.shape)
rawdata.head()
# ID vol sample: 'p22061-s001_barcode269931-subjectidCC0069-iridCR141-date20200605' 

In [ ]:
# Here i want to make it better readable what single entries mean (i split up that identifier string. .. )
columns_order = ['sample_id', 'barcode', 'subjectid', 'irid', 'date'] + list(rawdata.columns)
rawdata[['sample_id', 'barcode', 'subjectid', 'irid', 'date']] = rawdata['gene_id'].str.split('-', expand=True)
## add the first 4 chars of the barcode columns to the first one, as it is visible in the .xlsx file that it is supposed to be part of the sample_id
prefixes = rawdata['barcode'].str[:4]
rawdata['sample_id'] = rawdata['sample_id'] + '_' + prefixes
rawdata = rawdata[columns_order]
rawdata.head()

In [ ]:
prefixes = {
  'subjectid': 'subjectid',
  'irid': 'irid',
  'date': 'date'
}
for column, prefix in prefixes.items():
  rawdata[column] = rawdata[column].str.replace(f'^{prefix}', '', regex=True)
rawdata['barcode'] = rawdata['barcode'].str[12:] # buts the sxxx_barcode prefix
rawdata.head()

In [ ]:
# Convert the date_column to datetime format so i can hopefully merge them
metadata['PBMC.Collection.Date'] = pd.to_datetime(metadata['PBMC.Collection.Date'], format='%m/%d/%y')
metadata['PBMC.Collection.Date'] = metadata['PBMC.Collection.Date'].dt.strftime('%Y%m%d')


In [ ]:
# I now try to create a new index to merge meta with rawdata
metadata['merge_index'] = metadata['PBMC.Collection.Date'] + metadata['Subject.ID']
print(metadata.head())

rawdata['merge_index'] = rawdata['date'] + rawdata['subjectid']
rawdata.head()

In [ ]:
combi_df = pd.merge(metadata, rawdata, on='merge_index', how='inner')
print(combi_df.shape)

## convert age 
combi_df['Age (years)'] = combi_df['Age at PBMC Collection (Months)'] / 12.
combi_df.head()

In [ ]:
combi_df.groupby('disStage').count()

In [ ]:
combi_df.to_csv(PATH_WIMMERS2023_WORK + '2023_Wimmers_combi_df.csv')

## As there was a confusion with PBMC vs. whole blood datatype (PBMC stated in col header and whole blood in https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE239787) i also want to import the raw counts from GEO 


In [ ]:
raw_counts_GEO_df = pd.read_csv(PATH_WIMMER2023_GEO_RAW, sep="\t")
print(raw_counts_GEO_df.shape)
raw_counts_GEO_df.head()

In [ ]:
import sys
import geo_utils
from geo_utils import read_prefixes_from_metadata_txt
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))

metadata_GEO_df = read_prefixes_from_metadata_txt(PATH_WIMMER2023_GEO_METADATA)
metadata_GEO_df[['ID', 'Status']] = metadata_GEO_df['title'].str.replace('Subject ', '').str.split(' ', expand=True)
print(metadata_GEO_df.shape)


In [ ]:
columns_metadata = ['geo_accession', 'source_name_ch1', 'instrument_model', 'library_strategy', 'platform_id', 'ID', 'Status',
                    'characteristics_ch1.2', 'characteristics_ch1.3', 'characteristics_ch1.4', 'contact_city', 'description']
metadata_GEO_df_reduced_metadata = metadata_GEO_df[columns_metadata]
metadata_GEO_df_reduced_metadata.head()

In [ ]:
raw_counts_GEO_df.T 
raw_counts_transposed_df = raw_counts_GEO_df.set_index('x[, 1]').T.reset_index().rename(columns={'index': 'key'})
raw_counts_transposed_df.head()

In [ ]:
metadata_GEO_df_reduced_metadata = metadata_GEO_df_reduced_metadata.rename(columns={'description':'key'})
merged_df_GSE239787 = pd.merge(metadata_GEO_df_reduced_metadata, raw_counts_transposed_df, on='key')
print(merged_df_GSE239787.shape)
merged_df_GSE239787.head()

In [ ]:
merged_df_GSE239787[merged_df_GSE239787['ID']=="CC0014"]

In [ ]:
merged_df_GSE239787.groupby('Status').count()

In [ ]:
merged_df_GSE239787.to_csv(PATH_WIMMERS2023_GEO)